In [3]:
import pandas as pd
import numpy as np
import re

In [7]:
"""
ЗАВАНТАЖЕННЯ ФАЙЛУ З BIPRO.PROZORRO ІЗ ЗАВЕРШЕНИМИ ЗАКУПІВЛЯМИ МІСЬКИХ РАД
"""


df_raw = pd.read_csv('22_regions.csv', sep=';', low_memory = False)
df_raw = df_raw.drop(0)
df_raw['ОписаниеТендера'] = df_raw['ОписаниеТендера'].fillna(value='Пусто')
years_to_drop = ['UA-2015', 'UA-2016']
df_raw = df_raw[~df_raw['ИдентификаторЛота'].str.contains('|'.join(years_to_drop))]

З 2017 року: 311085


In [8]:
#набір ключових слів-фільтрів для відділення шкільних закупівель
filter_keywords = ['зошщ', 'навчально-виховн' ,'школ', 'шкіль', 'гімназ', 'ліцей', 'ліцея', 'зош', 'знз', 'сш', 'осш', 'колегіум', 'нвк', 'увк', 'снвк', 'загальноосвітн', 'сзш', 'виховн', 'днз', 'садок', 'дошкільн', 'садка']

#набір ключових слів-фільтрів для кожної категорії окремо
filter_shkola = ['школ', 'шкіль',  'гімназ',  'ліцей', 'ліцея', 'зош', 'знз', 'сш', 'колегіум', 'нвк', 'увк', 'снвк', 'загальноосвітн', 'сзш', 'виховн', 'осш']
filter_shkola_name = 'Школа'

filter_sadik = ['днз', 'ДНЗ', 'садок', 'дошкільн', 'садка', 'ясла', 'ясел']
filter_sadik_name = 'ДНЗ'

filter_sport = ['ДЮСШ', 'юнацька спортив', 'дюсш', 'СДЮСШОР', 'дитячо-спортивна']
filter_sport_name = 'ДЮСШ'

filter_music = ['музич', "дмш"]
filter_music_name = 'Музична школа'

filter_art = ['мистецтв', 'дхш']
filter_art_name = 'Школа мистецтв'

filter_hud = ['худож']
filter_hud_name = 'Художня школа'

filter_poza = ['позашкільн', 'міжшкільн', 'МСВЗШ', 'спеціальна вечірня']
filter_poza_name = 'Позашкільне'

filter_inshe = 'Інше'

nezamovnik = 'Незамовники'

In [13]:
"""
ФІЛЬТРУВАННЯ ШКІЛЬНИХ ЗАКУПІВЕЛЬ
"""


columns_to_check = ['Тендер', 'Организатор', 'ОписаниеТендера', 'Лот']
filtered = []

for column in columns_to_check:
    filtered.append(df_raw[df_raw[column].str.contains('|'.join(filter_keywords),case=False)])

df_filtered = pd.concat(filtered).drop_duplicates()
print("З-поміж",len(df_raw), "знайдено",len(df_filtered))    

In [18]:
"""
РОЗДІЛЕННЯ ШКІЛЬНИХ ЗАКУПІВЕЛЬ НА ТІ, ДЕ ШКОЛА Є ЗАМОВНИКОМ І ТІ, ДЕ ДЕПАРТАМЕНТ ЧИ ІНШИЙ ОРГАН КУПУЄ ЩОСЬ ДЛЯ ШКОЛИ
"""


df_filtered['Тип'] = pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_music), case = False), filter_music_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_art), case = False), filter_art_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
                            pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
                                         nezamovnik)))))))

print(df_filtered['Тип'].value_counts())

Незамовники       22770
Школа              8848
ДНЗ                1639
ДЮСШ               1258
Позашкільне         759
Школа мистецтв      497
Музична школа       427
Художня школа        55
Name: Тип, dtype: int64


In [37]:
"""
РЕГУЛЯРНІ ВИРАЗИ
"""


reg_number = '(№\s*\d+)' #витягує число перед яким є символ №
reg_quotes = """["'«](.*?)["'»]{1}?""" #витягує текст в лапках трьох видів
lot_regex = '(((\DЗОШ\D)|(\Dзош\D)|(\DСШ\D)|(\DДНЗ\D)|(\DДНЗ\D)).(\d+))' #витягує номер поруч з ключовими словами
reg_city = "(м\.\w+)" # витягує назву міста після "м."

In [20]:
"""
СТВОРЕННЯ БАЗИ ШКІЛ-ЗАМОВНИКІВ І ПОДІЛ ЇЇ НА ДВІ БАЗИ: З І БЕЗ НОМЕРУ

ВІДБИРАННЯ З БАЗИ ШКІЛ ТИХ, ДЕ ВОНИ Є ЗАМОВНИКАМИ І ПРИБИРАННЯ ВСІХ "ПОЗАШКІЛЬНИХ" ЗАКУПІВЕЛЬ
ВИЗНАЧЕННЯ НОМЕРУ ЗА НАЯВНІСТЮ СИМВОЛА № І СТВОРЕННЯ ПЕРШОЇ БАЗИ ШКІЛ-ЗАМОВНИКІВ df_zamovnik_number
ДОДАТКОВЕ ФІЛЬТРУВАННЯ РЯДКІВ ЗА НАЗВОЮ В ЛАПКАХ І СТВОРЕННЯ ОКРЕМОЇ БАЗИ df_zamovnik_name
"""


df_zamovnik_number = df_filtered[(df_filtered['Тип'] != nezamovnik) & (df_filtered['Тип'] != filter_poza_name)]


df_zamovnik_number['number'] = df_zamovnik_number['Организатор'].str.extract(reg_number, expand = False)


df_zamovnik_name = df_zamovnik_number[df_zamovnik_number['number'].isnull()]
df_zamovnik_number = df_zamovnik_number[df_zamovnik_number['number'].notnull()]


df_zamovnik_name['number'] = df_zamovnik_name['Организатор'].str.extract(reg_quotes, expand = False)


#Заповнити колонку Тип значенням "Організатор" там, де не знаходить ні номеру, ні назви школи в лапках
df_zamovnik_name['Тип'] = np.where(df_zamovnik_name['number'].isnull(), df_zamovnik_name['Организатор'], df_zamovnik_name['Тип'])

print("Кількість шкіл-замовників без номеру:", len(df_zamovnik_name))
print("Кількість шкіл-замовників з номером:", len(df_zamovnik_number))

Кількість шкіл-замовників без номеру: 2943
Кількість шкіл-замовників з номером: 9781


C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [21]:
"""
СТВОРЕННЯ БАЗИ ШКІЛ НЕ-ЗАМОВНИКІВ df_department

"""


df_department = df_filtered[df_filtered['Тип'] == nezamovnik]

#присвоєння школам-замовникам типу
df_department['Тип'] = pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_music), case = False), filter_music_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_music), case = False), filter_music_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_music), case = False), filter_music_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_music), case = False), filter_music_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_art), case = False), filter_art_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_art), case = False), filter_art_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_art), case = False), filter_art_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_art), case = False), filter_art_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
    pd.np.where(df_department["Тендер"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
    pd.np.where(df_department["ОписаниеТендера"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
    pd.np.where(df_department["Адрес поставки"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
    pd.np.where(df_department["Лот"].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
    filter_inshe))))))))))))))))))))))))))))

print("Позашкільних закладів:", len(df_department[df_department['Тип'] == 'Позашкільне']))

df_department = df_department[df_department['Тип'] != 'Позашкільне']

#Прибрати все позашкільне
print("Всього закупівель департаменту:", df_department.shape[0])
print("За типом:", df_department['Тип'].value_counts())

#Створити колонку з номерами. Поетапне фільтрування в трьох колонках за номером, а потім - назвами в дужках
df_department['number'] = df_department['Тендер'].str.extract(reg_number, expand = False)
df_department['number'] = np.where(df_department['number'].isnull(), 
                                               df_department['Лот'].str.extract(reg_number, expand = False),
                                               df_department['number'])
df_department['number'] = np.where(df_department['number'].isnull(), 
                                               df_department['ОписаниеТендера'].str.extract(reg_number, expand = False),
                                               df_department['number'])
df_department['number'] = np.where(df_department['number'].isnull(), 
                                               df_department['Тендер'].str.extract(reg_quotes, expand = False),
                                               df_department['number'])
df_department['number'] = np.where(df_department['number'].isnull(), 
                                               df_department['Лот'].str.extract(reg_quotes, expand = False),
                                               df_department['number'])
df_department['number'] = np.where(df_department['number'].isnull(), 
                                               df_department['ОписаниеТендера'].str.extract(reg_quotes, expand = False),
                                               df_department['number'])

# створення бази без номерів (№ і назви в лапках)
df_department_problem = df_department[df_department['number'].isnull()]

df_department_number = df_department[df_department['number'].notnull()]

df_department_problem['number'] = df_department_problem['Лот'].str.extract(lot_regex, expand = False)
df_department_problem['number'] = pd.np.where(df_department_problem['number'].isnull(), 
                                df_department_problem['ОписаниеТендера'].str.extract(lot_regex, expand = False)[0],
                                df_department_problem['number'])
df_department_problem = df_department_problem[~df_department_problem['number'].str.contains('-', regex = False, na = False)] #прибирання всіх номерів, де числа є від'ємними (помилковими)
df_department_problem['number'] = '№' + df_department_problem['number'].str.extract('(\d+)', expand = False)

print("Спроба визначити в",len(df_department_problem))

df_department_problem = df_department_problem[df_department_problem['number'].notnull()]

print("Вдалось визначити",len(df_department_problem))

C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Позашкільних закладів: 225
Всього закупівель департаменту: 22545
За типом: Школа             11558
ДНЗ                9409
ДЮСШ                988
Музична школа       367
Школа мистецтв      139
Художня школа        84
Name: Тип, dtype: int64


C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Спроба визначити в 2104
Вдалось визначити 179
Ще залишилось опрацювати 20438


# Виправлення помилок

In [22]:
# список лєвачьків-замовників, яких треба видалити
bad_zamovnik_names = ['Комунальне підприємство "Шкільне" Чернігівської міської ради', 
                      'Хмельницький міський центр комплексної реабілітації дітей з інвалідністю "Школа життя"', 
                      'ХАРКІВСЬКА ШКОЛА МИСТЕЦТВ ДЛЯ ПІДЛІТКІВ ТА ДОРОСЛИХ',
                      'Київська Академія мистецтв',
                      'Запорізьке вище професійне училище моди і стилю',
                      'Сумський хіміко-технологічний центр професійно -технічної освіти']

print("Було:", len(df_zamovnik_name))
df_zamovnik_name = df_zamovnik_name[~df_zamovnik_name['number'].isin(bad_zamovnik_names)]
print("Залишилось:", len(df_zamovnik_name))

# Перейменування проблемних назв
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('Київський політехнічний інститут', 'Технічний ліцей НТУУ КПІ')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('НВК', 'Вальдорфська НВК')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('МІСЬКА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ІЗ ЗИМОВИХ ВИДІВ СПОРТУ', 'ДЮСШ із зимових видів спорту')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('ЛІЦЕЙ МИТНО-ПОДАТКОВОЇ СПРАВИ З ПОСИЛЕНОЮ ВІЙСЬКОВО-ФІЗИЧНОЮ ПІДГОТОВКОЮ ПРИ УНІВЕРСИТЕТІ МИТНОЇ СПРАВИ ТА ФІНАНСІВ', 'Ліцей митно-податкової справи')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('Вінниця', 'Міська комплексна ДЮСШ "Вінниця"')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('МІСЬКА СПЕЦІАЛІЗОВАНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ОЛІМПІЙСЬКОГО РЕЗЕРВУ З ВОДНИХ ВИДІВ СПОРТУ ЯНИ КЛОЧКОВОЇ', 'ДЮСШ Олімпійського резерву з водних видів спорту Яни Клочкової')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('СДЮСШОР з плавання ДМГО СК ', 'ДЮСШ "Метеор"')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('СПЕЦІАЛІЗОВАНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ОЛІМПІЙСЬКОГО РЕЗЕРВУ З ВЕСЛУВАННЯ ', 'ДЮСШ Олімпійського резерву з веслування')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('ДЮСШ ', 'Атлет')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('КПНЗ ', 'з водних та веслувальних видів спорту')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('Полтавська дитячо-спортивна школа ', 'Ворскла')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('Міська - дитячо -юнацька спортивна школа з опорних видів спорту', 'з опорних видів спорту')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('СПЕЦІАЛІЗОВАНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ОЛІМПІЙСЬКОГО РЕЗЕРВУ ІМЕНІ ЛІТВАКА БОРИСА ДАВИДОВИЧА', 'ДЮСШ Олімпійського резерву Бориса Літвака')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('ДНІПРОВСЬКИЙ ЛІЦЕЙ ІНФОРМАЦІЙНИХ ТЕХНОЛОГІЙ ПРИ ДНІПРОВСЬКОМУ НАЦІОНАЛЬНОМУ УНІВЕРСИТЕТІ ІМЕНІ ОЛЕСЯ ГОНЧАРА', 'інформаційних технологій при дніпровському університеті Олеся Гончара')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('СПЕЦІАЛІЗОВАНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ОЛІМПІЙСЬКОГО РЕЗЕРВУ З ВОДНОГО ПОЛО', 'ДЮСШ Олімпійського резерву з водного поло')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('Комплексна дитячо-юнацька спортивна школа ', '"Суми"')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('СПЕЦІАЛІЗОВАНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА ОЛІМПІЙСЬКОГО РЕЗЕРВУ З КІННОГО СПОРТУ', 'ДЮСШ Олімпійського резерву з кінного спорту')
df_zamovnik_name['number'] = df_zamovnik_name['number'].replace('ЗОШ І ст.-ДНЗ', 'Пролісок')


# видалення лєвачьків
very_bad_words = ['будівництво', 'реконструкці', 'поcлуги', 'роботи', 'товари', 'відновлення', 'ремонт', 'меблі', 
                  'технічний', 'одяг', 'гравій', 'єкту',
                 'килимові', 'встановлення', 'території', 'марафон', 'правила', 'інформаційний',
                 'територія', 'завершення', 'єкт', 'улаштування', 'діти', 'водопостачання', 'цпотд',
                  'шкільна', 'техніки', 'техніка', 'документації', 'приміщень', 'УВК-А8-СН80', 
                  'ХВПУ СД', 'будівля', 'масштабування', 'машини', 'офісні', 'cruise', 'ЛВПУХТ',
                  'технічного', 'прилади', 'львівтеплоенерго', 'інвентар', 'ЦПО ІТПД', 'крем', 'грантрайтингу',
                  'устаткування', 'ЦПОІТПД', 'MyLaB', 'будівлі', 'обладнання', 'інвентар', 'приладу', 
                  'вироби', 'Seven', 'VACUUM', 'spase', '4545', '452', '3911', 'Hitachi', 'Puppy', 'Women Run', 'ІППОН',
                  '2-А', 'комплексів', 'сорочки', 'Тірас', 'аналіза', 'Г-1', 'адрес', 'щити', 'друку', 'розділах', 
                  'професійн', 'чаші', 'Послуги', 'користування', 'єрів', 'бюджетної', 'покрівлі', 'Апаратура', 
                  'Вітаю,', '7124', 'Микільська Борщагівка', 'Абетка українська', 'Біомед', 
                  'підлоги', 'мужності', 'електросилові', 'шахів', 'викладачів', 'пешка', 
                  'футбольні', 'школярів', 'С.Л.Г.', 'ВУВКГ м. Херсона', 'ОРТ-Алєф', 'мікрорайон', 'програми', 
                  'ПЦПТО', 'Сон. апельсин', 'будівельними', 'словник', 'А-2', 'Безпечна школа', 'цінностей',
                  'ПОВЕРХОВА', '4542', '1,2,3', 'приміщення', 'Електродинаміка', 'ігрового', 'фінансування', 'радості', 
                  'житло', 'центр хореографічного', 'енерго', 'приладдя', 'документація', 'компресор',
                  'інструменти', 'взуття', 'Ен', 'Нова Українська Школа', 'серванти', 'Вільно/Зайнято', 'матеріали', 
                  'Мікроскопи', 'української школи', 'персональних', 'проект', 'столи', 'Фарби', 'Форт-17', 'Іграшки'                  
                 ]

bad_isin = ['Ліцей', 'ТМО', 'Ушастый нянь', 'Харків – спортивна столиця', 'Хускварна', 'єва - п', 'Худоджня школа', 'Художня школа']

df_department_number = df_department_number[~df_department_number['number'].str.contains('|'.join(very_bad_words), case=False, na = False)]
df_department_number = df_department_number[~df_department_number['number'].isin(bad_isin)]

# поштучне перейменування
df_department_number['number'] = pd.np.where(df_department_number["Лот"].str.contains('Сумська класична гімназія', case = False), 
                                           'класична гімназія',
    pd.np.where(df_department_number["Лот"].str.contains('Лінгвістичної гімназії', case = False), 'Лінгвістична гімназія',
    pd.np.where(df_department_number["Лот"].str.contains('СДЮСШОР з фехтування', case = False), 'з фехтування',
    pd.np.where(df_department_number["Лот"].str.contains('ДЮСШ Дерюгіних', case = False), 'Дерюгіних',
                df_department_number['number'])))) 

#створення окремої бази без №
df_department_name = df_department_number[~df_department_number['number'].str.contains('№')]
df_department_name['number'] = df_department_name['number'].replace(to_replace ='^\s', value = '', regex = True) 
df_department_name['number'] = df_department_name['number'].str.capitalize()

df_department_name['number'] = df_department_name['number'].replace('Дитячо-юнацька спортивна школа ', 'Авангард')
df_department_name['number'] = df_department_name['number'].replace('Дитячо-юнацька спортивна школа ', 'Авангард')
df_department_name['number'] = df_department_name['number'].replace('Волошка', '№57')
df_department_name['number'] = df_department_name['number'].replace('Ясла-садок', '№106')
df_department_name['number'] = df_department_name['number'].replace('Коммунарівець', 'Комунарівець')
df_department_name['number'] = df_department_name['number'].replace('Комунаровець', 'Комунарівець')
df_department_name['number'] = df_department_name['number'].replace('Стрімкий м', 'Стрімкий м’яч')
df_department_name['number'] = df_department_name['number'].replace('Стрімкий мяч', 'Стрімкий м’яч')
df_department_name['number'] = df_department_name['number'].replace('Садок -школа', 'Веселка')
df_department_name['number'] = df_department_name['number'].replace('Садок-школа', 'Дзвіночок')
df_department_name['number'] = df_department_name['number'].replace('Дитяча музична школа ', '№15')
df_department_name['number'] = df_department_name['number'].replace('Днз ', '№')
df_department_name['number'] = df_department_name['number'].replace('Дошкільний навчальний заклад-Загальноосвітній навчальний заклад ', "Сузір'я")
df_department_name['number'] = df_department_name['number'].replace('Навчально-виховний комплекс ', 'Вальдорфський нвк')
print("Закупівель департамену без номеру",len(df_department_name))

#створення окремої бази, де зустрічається №
df_department_number = df_department_number[df_department_number['number'].str.contains('№')]

df_department_number['number'] = df_department_number['number'].str.replace(u'\xa0', u' ')
df_department_number['number'] = df_department_number['number'].str.replace(u'\u202f', u' ')
df_department_number['number'] = df_department_number['number'].str.replace(u'\n', u' ')
df_department_number['number'] = df_department_number['number'].str.replace(' ', '')
df_department_number = df_department_number[df_department_number['number'].map(len) < 5]

print("Закупівель департамену з номером",len(df_department_number))

Було: 2943
Залишилось: 2828


C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

Закупівель департамену без номеру 637
Закупівель департамену з номером 18999


In [1434]:
# для перевірки якості визначення типу шкіл-організаторів
# pd.options.display.max_colwidth = 100
# df_zamovnik_number[['Организатор','Тип']]

# df_zamovnik_name[['Организатор', 'number']]

# поштучна перевірка
# df_zamovnik_name[df_zamovnik_name['number'] == 'Метеор']

# print("Для перевірки. Назви шкіл без номеру:",df_zamovnik_number['number'].unique())
# print("Для перевірки. Назви шкіл без номеру:",df_zamovnik_name['number'].unique())

s = df_department_name['number'].unique().tolist()
s.sort()
s

# df_department_name[df_department_name['number'].str.contains('Днз ', case = False, na=False)]

"""
проблєма яку треба вирішити: 'ДНЗ – ЗНЗ «Інтел' залишити все що після «

df2 = df_department_name
df2['number'] = pd.np.where(df2["number"].str.contains('«', case = False, na = False), 
                            df2['number'].str.extract(lot_regex, expand = False)[0], df2['number'])

df_department_name['number'] = df_department_name['number'].str.extract('(\d+)', expand = False)

# df_department2 = df_department
# regex_after_quote = '«.*'
# df_department2['number'] = df_department2['number'].str.split('\«', 1)
# df_department2[df_department2['number'].str.contains('інтел', case = False, na=False)]

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Сума договору (згідно договору),Адрес поставки,АдресОрганизатора,Главный орган,Тип,number
85882,UA-2017-11-14-000649-c-L1,UA-2017-11-14-000649-c,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,23100,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",ДК 021-2015 (CPV) 37415000-0 - Легкоатлетичний інвентар(Спис для метання тренувальний Polanik W-...,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,20740,вул. Макарова 27А,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор
85883,UA-2017-06-08-000370-c-L1,UA-2017-06-08-000370-c,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,20000,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",ДК 021-2015 (CPV)37415000-0 - Легкоатлетичний інвентар: Молот змагальний Polanik Польша : Рм 3/9...,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,13900,вул. Макарова 27А,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор
85994,UA-2017-06-08-000357-c-L1,UA-2017-06-08-000357-c,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,25000,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",ДК 021-2015 (CPV) 37415000-0 - Легкоатлетичний інвентар(Спис для метання тренувальний Polanik W-...,ДК 021-2015 (CPV) 37410000-5 - Інвентар для спортивних ігор на відкритому повітрі,17899,вул. Макарова 27А,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор
150856,UA-2017-06-08-001571-b-L1,UA-2017-06-08-001571-b,"Код ДК 021-2015 (CPV)\t 92620000-3 - Послуги, пов’язані зі спортом",30000,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",послуги тренажерного залу,"Код ДК 021-2015 (CPV)\t 92620000-3 - Послуги, пов’язані зі спортом",30000,вул. Макарова 27А,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор
225242,UA-2018-05-15-000544-b-L1,UA-2018-05-15-000544-b,"Код ДК 021-2015 (CPV)\t 92620000-3 - Послуги, пов’язані зі спортом",24990,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",послуги тренажерного залу,"Код ДК 021-2015 (CPV)\t 92620000-3 - Послуги, пов’язані зі спортом",24990,вул. Макарова 27А,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор
360594,UA-2019-02-04-000398-c-L1,UA-2019-02-04-000398-c,"Послуги, пов'язані зі спортом (послуги тренажерного залу)",25000,"КПНЗ СДЮСШОР з легкої атлетики ""Метеор"" ДМР",Пусто,"Послуги, пов'язані зі спортом (послуги тренажерного залу)",24970,-,віулиця Макарова 27А,Дніпровська міська рада,ДЮСШ,Метеор


In [23]:
"""
Створення однієї бази всіх шкіл df_full
 
"""


frames_zamovnik = [df_zamovnik_name, df_zamovnik_number]
df_zamovnik = pd.concat(frames_zamovnik)

frames_department = [df_department_number, df_department_name, df_department_problem]
df_department = pd.concat(frames_department)

df_zamovnik['Замовник'] = "Так"
df_department['Замовник'] = "Ні"

frames_full = [df_zamovnik, df_department]
df_full = pd.concat(frames_full).drop_duplicates(subset='ИдентификаторЛота')

# Створити нову колонку, обєднавши "Тип" і номер закладу
df_full = df_full.rename(columns={'Главный орган':'gol_organ'})
df_full['number'] = np.where(df_full['number'].str.contains('№'), 
                                               df_full['number'].str.replace('№ ', '№'),
                                               df_full['number'])

df_full['Заклад'] = np.where(df_full['number'].str.contains('№'), 
                                               df_full['Тип'] + " " + df_full['number'],
                                               df_full['number'])

print(len(df_full))
print(df_full.columns)

32320
Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот',
       'Сума договору (згідно договору)', 'Адрес поставки',
       'АдресОрганизатора', 'gol_organ', 'Тип', 'number', 'Замовник',
       'Заклад'],
      dtype='object')


In [45]:
"""
Створення бази відповідників Головний орган і місто

Завантаження зовнішньої бази шкіл з адресами

"""

regions = ['Суми', 'Київ', 'Полтава', 'Харків', 'Чернігів', 'Вінниця', 'Львів', 'Миколаїв', 'Дніпро',
           'Чернівці', 'Запоріжжя', 'Одеса', 'Черкаси', 'Херсон', 'Рівне', 'Тернопіль', 'Ужгород', 
           'Луцьк', 'Івано-Франківськ', 'Житомир', 'Кропивницький', 'Хмельницький']

gol_organ = ['Сумська міська рада',
 'КМДА',
 'Полтавська міська рада',
 'Харківська міська рада',
 'Чернігівська міська рада',
 'Вінницька міська рада',
 'Львівська міська рада',
 'Миколаївська міська рада',
 'Дніпровська міська рада',
 'Чернівецька міська рада',
 'Запорізька міська рада',
 'Одеська міська рада',
 'Черкаська міська рада',
 'Херсонська міська рада',
 'Рівненська міська рада',
 'Тернопільська міська рада',
 'Ужгородська міська рада',
 'Луцька міська рада',
 'Івано-Франківська міська рада',
 'Житомирська міська рада',
 'Кропивницька міська рада',
 'Хмельницька міська рада']

# Створення таблиці з головним органом і відповідним містом
city_organ = pd.DataFrame(list(gol_organ), columns=['gol_organ'])
city_organ['city'] = pd.Series(regions)
city_organ = city_organ[['gol_organ', 'city']]

cities_raw = pd.read_excel('schools&address.xls')
print(cities_raw.columns)

cities_raw['city'] = cities_raw['address'].str.extract(reg_city, expand = False)
cities_raw['city'] = cities_raw['city'].str.replace('м\.','')
cities_raw['city'] = cities_raw['city'].str.replace('Івано','Івано-Франківськ')

cities = cities_raw[cities_raw['city'].notnull()]

cities['number'] = cities.school.str.extract(reg_number, expand = False)
cities['number'] = np.where(cities['number'].isnull(), cities['school'].str.extract(reg_quotes, expand = False), cities['number'])
cities['number'] = np.where(cities['number'].str.contains('№'), cities['number'].str.replace(' ',''), cities['number'])

cities['type'] = pd.np.where(cities['school'].str.contains('|'.join(filter_sport), case = False), filter_sport_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_music), case = False), filter_music_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_art), case = False), filter_art_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_hud), case = False), filter_hud_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_poza), case = False), filter_poza_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_sadik), case = False), filter_sadik_name,
                            pd.np.where(cities['school'].str.contains('|'.join(filter_shkola), case = False), filter_shkola_name,
                                         filter_inshe)))))))

# Створити нову колонку, обєднавши "Тип" і номер закладу
cities['Заклад'] = np.where(cities['number'].str.contains('№'), 
                                               cities['type'] + " " + cities['number'],
                                               cities['number'])

cities['Заклад'] = np.where(cities['Заклад'].isnull(), cities['school'], cities['Заклад'])

cities = cities[cities.city.str.contains('|'.join(regions), case=False)]
cities['city_school'] = cities['city'] + ' ' + cities['Заклад']
cities = cities[['city_school', 'address']]
cities.sample()

Index(['school', 'address'], dtype='object')


C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,city_school,address
12635,Харків Школа №23,"Україна, Харківська область, м.Харків, Московс..."


In [42]:
df_full.sample(5)

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Сума договору (згідно договору),Адрес поставки,АдресОрганизатора,gol_organ,Тип,number,Замовник,Заклад
173646,UA-2018-09-20-002164-c-L1,UA-2018-09-20-002164-c,Роботи з монтажу системи пожежної сигналізації...,197844.71,"КУ ""КЗ ""НВО № 32 ""СПЕЦІАЛІЗОВАНА ЗАГАЛЬНООСВІТ...",Пусто,Роботи з монтажу системи пожежної сигналізації...,197844.71,"вул. Глинки,1","вулиця Глинки, 1",Кропивницька міська рада,Школа мистецтв,№32,Так,Школа мистецтв №32
324080,UA-2019-06-19-002558-c-L1,UA-2019-06-19-002558-c,Капітальний ремонт приміщень Івано-Франківсько...,245300,Департамент освіти та науки Івано-Франківської...,Пусто,Капітальний ремонт приміщень Івано-Франківсько...,238002,"вулиця Національної Гвардії, 13","вулиця Кардинала Любомира Гузара, 15",Івано-Франківська міська рада,Школа,№12,Ні,Школа №12
112003,UA-2017-04-27-000284-b-L1,UA-2017-04-27-000284-b,"Поточний ремонт системи опалення в НВК ""Любисток""",120000,УПРАВЛІННЯ ОСВІТИ ЧЕРНІВЕЦЬКОЇ МІСЬКОЇ РАДИ,Пусто,"Поточний ремонт системи опалення в НВК ""Любисток""",88060,Героїв Майдану 176,Героїв Майдану 176,Чернівецька міська рада,Школа,Любисток,Ні,Любисток
252600,UA-2018-02-22-001333-b-L1,UA-2018-02-22-001333-b,Поточний ремонт внутрішньої мережі холодного в...,17000,Управління освіти Дніпровської районної в міст...,Умови оплати: відтермінування платежу до 90 ка...,Поточний ремонт внутрішньої мережі холодного в...,16006.69,"вул. М. Кибальчича, 5","пр-т Миру, 6-А",КМДА,Школа,№224,Ні,Школа №224
347383,UA-2019-03-26-001120-c-L1,UA-2019-03-26-001120-c,"Будівельно-монтажні роботи по об'єкту ""Капітал...",430662.8,Управління капітального будівництва Міської ради,"Будівельно-монтажні роботи по об'єкту ""Капітал...","Будівельно-монтажні роботи по об'єкту ""Капітал...",430662.8,"вул. Юрія Коваленка, 9-а","вул. Велика Перспективна, 41, каб. 410",Кропивницька міська рада,Школа,№18,Ні,Школа №18


In [48]:
"""
Додавання до головної бази: 
                            посилань на закупівлі
                            адрес шкіл з зовнішньої бази 
                            
                            різних варіантів адреси школи
 
"""


# колонки назвою школи та міста
df_full_connector = pd.merge(df_full, city_organ, on = 'gol_organ')
df_full_connector['city_school'] = df_full_connector['city'] + ' ' + df_full_connector['Заклад']

# колонка з альтернативною адресою школи 'address'
df_full_connector = df_full_connector.merge(cities, on = 'city_school', how = 'left')

df_full_connector['address_zamovnik_or_postavka'] = np.where(df_full_connector['Замовник'] == 'Так',
                                        df_full_connector['АдресОрганизатора'],
                                        df_full_connector['Адрес поставки'])

# колонки з адресою замовника чи поставки та містом
df_full_connector['city_zamovnik_or_postavka'] = df_full_connector['city'] + ' ' + df_full_connector['address_zamovnik_or_postavka']

# колонки з посиланнями на закупівлі DOZORRO
df_full_connector['link'] = ('https://dozorro.org/tender/' + df_full_connector['Идентификатор'])

# Заповнення невідомих адрес "Невідомо"
df_full_connector['address'][df_full_connector.isna().any(axis = 1)] = 'Невідомо'
df_full_connector['address_zamovnik_or_postavka'][df_full_connector.isna().any(axis = 1)] = 'Невідомо'
df_full_connector['city_zamovnik_or_postavka'][df_full_connector.isna().any(axis = 1)] = 'Невідомо'

# Перетворення колонки з сумою договору на числовий формат
df_full_connector["Сума договору (згідно договору)"] = pd.to_numeric(df_full_connector["Сума договору (згідно договору)"])

print(df_full_connector.columns)

print("Унікальних закладів:", len(df_full_connector['city_school'].unique().tolist()))
# df_full_connector['number'].unique().tolist()

Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот',
       'Сума договору (згідно договору)', 'Адрес поставки',
       'АдресОрганизатора', 'gol_organ', 'Тип', 'number', 'Замовник', 'Заклад',
       'city', 'city_school', 'address', 'address_zamovnik_or_postavka',
       'city_zamovnik_or_postavka', 'link'],
      dtype='object')
Унікальних закладів: 3955


In [240]:
df_full_connector.sample(10)

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Сума договору (згідно договору),Адрес поставки,АдресОрганизатора,...,Тип,number,Замовник,Заклад,city,city_school,address,address_zamovnik_or_postavka,city_zamovnik_or_postavka,link
25169,UA-2018-05-04-000420-a-L1,UA-2018-05-04-000420-a,Виготовлення проектно-кошторисної документації...,64628.09,Черкаська загальноосвітня школа І-ІІІ ступенів...,Згідно договору,Виготовлення проектно-кошторисної документації...,64628.09,"вул.Бидгощська, 1","вул. Бидгощська, 1",...,Школа,№24,Так,Школа №24,Черкаси,Черкаси Школа №24,"Україна, Черкаська область, м.Черкаси, Бидгощс...","вул. Бидгощська, 1","Черкаси вул. Бидгощська, 1",https://dozorro.org/tender/UA-2018-05-04-000420-a
22749,UA-2018-10-17-000019-a-L1,UA-2018-10-17-000019-a,Поточний ремонт систенми опалення дошкільного...,50997,Управління освіти Рівненського міськвиконкому,Пусто,Поточний ремонт систенми опалення дошкільного...,50997.00,"вул. Р. Шухевича,28, ДНЗ №55","вул. Соборна, 30",...,ДНЗ,№55,Ні,ДНЗ №55,Рівне,Рівне ДНЗ №55,Невідомо,"вул. Р. Шухевича,28, ДНЗ №55","Рівне вул. Р. Шухевича,28, ДНЗ №55",https://dozorro.org/tender/UA-2018-10-17-000019-a
613,UA-2017-05-17-001133-b-L1,UA-2017-05-17-001133-b,Поточний ремонт по заміні окремих віконних та ...,197054.62,"Комунальна установа ""Центр фінансування та гос...",Пусто,Поточний ремонт по заміні окремих віконних та ...,197054.62,"м.Одеса, вул. Рішельевська, 18, ОСН №117","вул. Канатна, буд.134",...,Школа,№117,Ні,Школа №117,Одеса,Одеса Школа №117,"Україна, Одеська область, м.Одеса, Приморський...","м.Одеса, вул. Рішельевська, 18, ОСН №117","Одеса м.Одеса, вул. Рішельевська, 18, ОСН №117",https://dozorro.org/tender/UA-2017-05-17-001133-b
25764,UA-2017-11-01-000580-a-L1,UA-2017-11-01-000580-a,(45210000-2) Послуги з поточного ремонту примі...,65000,Департамент освіти та гуманітарної політики Че...,Пусто,(45210000-2) Послуги з поточного ремонту примі...,65000.00,"вул.Толстого, 73/1","вул. Гоголя, 251",...,ДНЗ,№57,Ні,ДНЗ №57,Черкаси,Черкаси ДНЗ №57,Невідомо,"вул.Толстого, 73/1","Черкаси вул.Толстого, 73/1",https://dozorro.org/tender/UA-2017-11-01-000580-a
5352,UA-2018-10-03-002924-c-L1,UA-2018-10-03-002924-c,Телевізійне й аудіовізуальне обладнання (мульт...,89800,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,КЗО НВК № 148,Телевізійне й аудіовізуальне обладнання (мульт...,89772.00,"КЗО НВК № 148, вул. Антоновича, 13","просп. Д.Яворницького 75,А",...,Школа,№148,Ні,Школа №148,Дніпро,Дніпро Школа №148,Невідомо,"КЗО НВК № 148, вул. Антоновича, 13","Дніпро КЗО НВК № 148, вул. Антоновича, 13",https://dozorro.org/tender/UA-2018-10-03-002924-c
5375,UA-2019-03-27-001996-b-L1,UA-2019-03-27-001996-b,"Комп’ютерне обладнання та приладдя (ноутбуки, ...",25554,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,"КЗО НВК № 28, проект № 18","Комп’ютерне обладнання та приладдя (ноутбуки, ...",25363.20,"КЗО НВК № 28, вул. Володі Дубініна, 12","просп. Д.Яворницького 75,А",...,Школа,№28,Ні,Школа №28,Дніпро,Дніпро Школа №28,Невідомо,"КЗО НВК № 28, вул. Володі Дубініна, 12","Дніпро КЗО НВК № 28, вул. Володі Дубініна, 12",https://dozorro.org/tender/UA-2019-03-27-001996-b
7228,UA-2018-12-11-002609-a-L1,UA-2018-12-11-002609-a,Придбання шкільних меблів (кабінетне обладнанн...,91000,школа І-ІІІ ступенів № 259 Деснянського району...,Пусто,Придбання шкільних меблів (кабінетне обладнанн...,91000.00,"проспект Володимира Маяковського, 21-Г","Київ-225, проспект Володимира Маяковського, 21-г",...,Школа,№259,Так,Школа №259,Київ,Київ Школа №259,"02225, м.Київ, проспект Маяковського, 21-Г","Київ-225, проспект Володимира Маяковського, 21-г","Київ Київ-225, проспект Володимира Маяковськог...",https://dozorro.org/tender/UA-2018-12-11-002609-a
17254,UA-2017-05-16-000342-a-L1,UA-2017-05-16-000342-a,код за ДК 021:2015 - 50530000-9 Послуги з ремо...,19382.76,КОМУНАЛЬНИЙ ЗАКЛАД “МІСЬКА СПЕЦІАЛІЗОВАНА ДИТЯ...,Послуги з поточного ремонту льодоприбиральног...,код за ДК 021:2015 - 50530000-9 Послуги з ремо...,19

In [242]:
df_full_connector[df_full_connector['Замовник']=='Ні']

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Сума договору (згідно договору),Адрес поставки,АдресОрганизатора,...,Тип,number,Замовник,Заклад,city,city_school,address,address_zamovnik_or_postavka,city_zamovnik_or_postavka,link
305,UA-2017-11-22-003177-a-L1,UA-2017-11-22-003177-a,Послуги з проведення технічної інвентарізації ...,16706.32,КУ Центр фінансування та господарської діяльно...,Пусто,Послуги з проведення технічної інвентарізації ...,16706.32,"вул. Висоцького, 14","вул. Отамана Головатого, 99",...,ДНЗ,№268,Ні,ДНЗ №268,Одеса,Одеса ДНЗ №268,Невідомо,"вул. Висоцького, 14","Одеса вул. Висоцького, 14",https://dozorro.org/tender/UA-2017-11-22-003177-a
306,UA-2017-07-20-000782-b-L1,UA-2017-07-20-000782-b,Капітальний ремонт : заміна окремих віконних т...,294592.8,КУ Центр фінансування та господарської діяльно...,Капітальний ремонт : заміна окремих віконних т...,Капітальний ремонт : заміна окремих віконних т...,294592.80,"вул. Давида Ойстраха, 15а","вул. Отамана Головатого, 99",...,Школа,№40,Ні,Школа №40,Одеса,Одеса Школа №40,"Україна, Одеська область, м.Одеса, Суворовськи...","вул. Давида Ойстраха, 15а","Одеса вул. Давида Ойстраха, 15а",https://dozorro.org/tender/UA-2017-07-20-000782-b
307,UA-2017-08-15-000251-b-L1,UA-2017-08-15-000251-b,Капітальний ремонт Одеської спеціалізованої шк...,283057.29,"Комунальна установа ""Центр фінансування та гос...",Пусто,Капітальний ремонт Одеської спеціалізованої шк...,283057.29,NaN,"вул. Канатна, буд.134",...,ДЮСШ,№121,Ні,ДЮСШ №121,Одеса,Одеса ДЮСШ №121,Невідомо,Невідомо,Невідомо,https://dozorro.org/tender/UA-2017-08-15-000251-b
308,UA-2017-05-12-000458-b-L1,UA-2017-05-12-000458-b,Здійснення авторського нагляду по об'єкту: Кап...,51840,Управління капітального будівництва Одеської м...,Здійснення авторського нагляду по об'єкту: Кап...,Здійснення авторського нагляду по об'єкту: Кап...,51840.00,"пров.Прохоровський,4","Комітетська, 10-А",...,ДНЗ,№237,Ні,ДНЗ №237,Одеса,Одеса ДНЗ №237,Невідомо,"пров.Прохоровський,4","Одеса пров.Прохоровський,4",https://dozorro.org/tender/UA-2017-05-12-000458-b
309,UA-2017-05-26-000328-c-L1,UA-2017-05-26-000328-c,постачання телевізорів LED HD 32” SONY в Одесь...,101496,КУ Центр фінансування та господарської діяльно...,постачання телевізорів LED HD 32” SONY в Одесь...,постачання телевізорів LED HD 32” SONY в Одесь...,101496.00,"Гайдара,52","вул. Генерала Петрова, будинок 22",...,ДНЗ,№217,Ні,ДНЗ №217,Одеса,Одеса ДНЗ №217,Невідомо,"Гайдара,52","Одеса Гайдара,52",https://dozorro.org/tender/UA-2017-05-26-000328-c
310,UA-2017-04-25-000257-a-L1,UA-2017-04-25-000257-a,Здійснення авторського нагляду по об’єкту:«Ре...,53557.2,Управління капітального будівництва Одеської м...,Здійснення авторського нагляду по об’єкту:«Ре...,Здійснення авторського нагляду по об’єкту:«Ре...,53557.20,"вул. Кустанайська, 3. Корпус ""А-2""","Комітетська, 10-А",...,Школа,№19,Ні,Школа №19,Одеса,Одеса Школа №19,"Україна, Одеська область, м.Одеса, Кустанайськ...","вул. Кустанайська, 3. Корпус ""А-2""","Одеса вул. Кустанайська, 3. Корпус ""А-2""",https://dozorro.org/tender/UA-2017-04-25-000257-a
311,UA-2017-05-18-000080-c-L1,UA-2017-05-18-000080-c,Капітальний ремонт по заміні віконних блок...,197051.8,КУ Центр фінансування та господарської діяльно...,Пусто,Капітальний ремонт по заміні віконних блок...,197051.80,"прооспект Маршала Жукова, 41а","вул. Академіка Корольова, 9",...,ДНЗ,№231,Ні,ДНЗ №231,Одеса,Одеса ДНЗ №231,Невідомо,"прооспект Маршала Жукова, 41а","Одеса прооспект Маршала Жукова, 41а",https://dozorro.org/tender/UA-2017-05-18-000080-c
312,UA-2017-12-18-001551-a-L1,UA-2017-12-18-001551-a,Розробка проектно-кошторисної документації об’...,200235.78,Управління капітального будівництва Одеської м...,Розробка проектно-кошторисної документації об’...,Розробка проектно-кошторисної документації об’...,200235.78,"пров. Прохоровський, 4.","Комітетська, 10-А",...,ДНЗ,№237,Ні,ДНЗ №237,Одеса,Одеса ДНЗ №237,Невідомо,"пров. Прохоровський, 4.","Одеса пров. Прохоровський, 4.",

# Підготовка баз

In [54]:
df_zakupki = df_full_connector.groupby('city_school')['link'].apply(lambda x: "%s"%', '.join(x))
df_zakupki = pd.DataFrame(df_zakupki)
df_zakupki.reset_index(inplace=True)
print(len(df_zakupki))
df_zakupki.sample()

3954


,city_school,link
64,Вінниця ДНЗ №11,https://dozorro.org/tender/UA-2019-03-18-000810-a


In [63]:
df_suma = df_full_connector.groupby(["city_school"]).sum()
print(len(df_suma))
df_suma.reset_index(inplace=True)
df_suma.sample()

3954


,city_school,Сума договору (згідно договору)
701,Запоріжжя ДНЗ №93,39866.0


In [127]:
df_address = pd.pivot_table(df_full_connector, index = ['city_school', 'address', 'city_zamovnik_or_postavka'])
del df_address['Сума договору (згідно договору)']
df_address.reset_index(inplace=True) 
print(len(df_address))
df_address.sample(10)

# print(len(df_address))
# df_address = df_address.drop_duplicates(subset='address1', keep='last', inplace=False)
# print(len(df_address))

# df_alternative_address = df_address.drop_duplicates(subset='address1', keep='first', inplace=False)

# del df_address['СуммаЛота']

13096


,city_school,address,city_zamovnik_or_postavka
6201,Київ Школа №314,"02095, м.Київ, вул. Княжий Затон, 7-А","Київ вул.Княжий Затон, 7а"
8356,Одеса ДНЗ №93,Невідомо,"Одеса Овыдыопольська дорога, 3"
6264,Київ Школа №327,"02152, м.Київ, ул. Серафімовича, 9-Б","Київ вул. Івана Миколайчука, 9-б у Дніпровсько..."
5570,Київ Школа №210,"04210, м.Київ, вул. Лайоша Гавро, 22-А","Київ Йорданська,22-А"
8841,Одеса Школа №60,"Україна, Одеська область, м.Одеса, Комітетська, 5","Одеса вул. Комітетська,5"
11862,Херсон Школа №16,"Україна, Херсонська область, м.Херсон, вул. Су...","Херсон Суворова,2"
554,Дніпро ДНЗ №297,Невідомо,"Дніпро вул. Калнишевського, 34 в"
1598,Запоріжжя ДНЗ №143,Невідомо,"Запоріжжя ДНЗ №143 - вул. Памірська, 93;"
233,Вінниця Музична школа №2,Невідомо,"Вінниця Стрілецька ,16"
5968,Київ Школа №27,"04080, м.Київ, пров. Цимлянський, 2","Київ вул. Ак. Туполєва, 20-Є"


In [83]:
df_address['address2'] = df_address['address'].str.split(',' , n=2, expand = True)[2]

df_address[df_address.city_school.str.contains('№10')]

,city_school,address,city_zamovnik_or_postavka,address2
2,Івано-Франківськ ДНЗ №10,Невідомо,"Івано-Франківськ вул. Миколайчука, 5",None
3,Івано-Франківськ ДНЗ №10,Невідомо,"Івано-Франківськ вулиця І.Миколайчука, 5",None
4,Івано-Франківськ ДНЗ №10,Невідомо,"Івано-Франківськ вулиця Івана Миколайчука, 5",None
77,Івано-Франківськ Школа №10,Невідомо,"Івано-Франківськ вулиця Вовчинецька, 196б",None
142,Вінниця ДНЗ №10,Невідомо,"Вінниця вул. Баженова, 30-А",None
143,Вінниця ДНЗ №10,Невідомо,"Вінниця вул. Пирогова,34а",None
238,Вінниця Школа №10,Невідомо,"Вінниця вул. Андрія Первозваного, 22",None
239,Вінниця Школа №10,Невідомо,Вінниця приміщенні ЗОШ № 10 по вул. Андрія Пер...,None
304,Дніпро ДНЗ №10,Невідомо,Дніпро навчальні заклади Замовника згідно Дисл...,None
305,Дніпро ДНЗ №102,Невідомо,"Дніпро КЗО НВК № 102, вул. Дзеркальна, 42А",None


In [213]:
address_test = df_address.copy()
# address_test['address3'] = ''
address_test.address = address_test.city_zamovnik_or_postavka.str.split().str.get(0) + ', ' + address_test['address'].str.rsplit(', ', expand = True, n = 3)[2] + ', ' + address_test['address'].str.rsplit(', ', expand = True, n = 3)[3]

address_test['address2'] = np.where(address_test.address.isnull(), address_test.city_zamovnik_or_postavka, address_test.address)
address_test

,city_school,address,city_zamovnik_or_postavka,address2
0,Івано-Франківськ ДНЗ №1,NaN,"Івано-Франківськ Незалежності, 6","Івано-Франківськ Незалежності, 6"
1,Івано-Франківськ ДНЗ №1,NaN,"Івано-Франківськ вулиця Вовчинецька, 9","Івано-Франківськ вулиця Вовчинецька, 9"
2,Івано-Франківськ ДНЗ №10,NaN,"Івано-Франківськ вул. Миколайчука, 5","Івано-Франківськ вул. Миколайчука, 5"
3,Івано-Франківськ ДНЗ №10,NaN,"Івано-Франківськ вулиця І.Миколайчука, 5","Івано-Франківськ вулиця І.Миколайчука, 5"
4,Івано-Франківськ ДНЗ №10,NaN,"Івано-Франківськ вулиця Івана Миколайчука, 5","Івано-Франківськ вулиця Івана Миколайчука, 5"
5,Івано-Франківськ ДНЗ №11,NaN,"Івано-Франківськ Незалежності, 6","Івано-Франківськ Незалежності, 6"
6,Івано-Франківськ ДНЗ №11,NaN,"Івано-Франківськ вулиця Академіка Сахарова, 32а","Івано-Франківськ вулиця Академіка Сахарова, 32а"
7,Івано-Франківськ ДНЗ №12,NaN,"Івано-Франківськ вул. Довга, 4","Івано-Франківськ вул. Довга, 4"
8,Івано-Франківськ ДНЗ №14,NaN,"Івано-Франківськ Марійки Підгірянки, 21","Івано-Франківськ Марійки Підгірянки, 21"
9,Івано-Франківськ ДНЗ №14,NaN,"Івано-Франківськ вулиця М.Підгірянки, 21","Івано-Франківськ вулиця М.Підгірянки, 21"


In [221]:
address_test.address.isna().sum()

7603

In [239]:
gr = address_test.groupby(['city_school', 'address2'])
gr.first()
# gr.get_group('Івано-Франківськ ДНЗ №1') 

address  \
city_school              address2                                                                                 
Івано-Франківськ ДНЗ №1  Івано-Франківськ Незалежності, 6                                                   NaN   
                         Івано-Франківськ вулиця Вовчинецька, 9                                             NaN   
Івано-Франківськ ДНЗ №10 Івано-Франківськ вул. Миколайчука, 5                                               NaN   
                         Івано-Франківськ вулиця І.Миколайчука, 5                                           NaN   
                         Івано-Франківськ вулиця Івана Миколайчука, 5                                       NaN   
Івано-Франківськ ДНЗ №11 Івано-Франківськ Незалежності, 6                                                   NaN   
                         Івано-Франківськ вулиця Академіка Сахарова, 32а                                    NaN   
Івано-Франківськ ДНЗ №12 Івано-Франківськ вул. Довга, 4                                                     NaN   
Івано-Франківськ ДНЗ №14 Івано-Франківськ Марійки Підгірянки, 21                                            NaN   
                         Івано-Франківськ вулиця М.Підгірянки, 21                                           NaN   
                         Івано-Франківськ вулиця М.Підгірянки,21                                            NaN   
Івано-Франківськ ДНЗ №15 Івано-Франківськ вул. Незалежності, 6                                              NaN   
                         Івано-Франківськ вулиця Нова, 19                                                   NaN   
Івано-Франківськ ДНЗ №16 Івано-Франківськ Вул. Короля Данила, 15а                                           NaN   
                         Івано-Франківськ Короля Данила, 15а                                                NaN   
                         Івано-Франківськ вул. Короля Данила,15 а                                           NaN   
                         Івано-Франківськ вулиця Короля Данила, 15                                          NaN   
                         Івано-Франківськ вулиця Короля Данила, 15а                                         NaN   
Івано-Франківськ ДНЗ №17 Івано-Франківськ вулиця Вовчинецька, 210                                           NaN   
Івано-Франківськ ДНЗ №18 Івано-Франківськ Івана Павла ІІ, 14                                                NaN   
                         Івано-Франківськ вул. Івана Павла ІІ, 14                                           NaN   
Івано-Франківськ ДНЗ №19 Івано-Франківськ вул. Незалежності, 6                                              NaN   
                         Івано-Франківськ вулиця Галицька, 101а                                             NaN   
Івано-Франківськ ДНЗ №2  Івано-Франківськ Бельведерська, 49А                                                NaN   
                         Івано-Франківськ Незалежності, 6                                                   NaN   
                         Івано-Франківськ вул. Незалежності, 6                                              NaN   
                         Івано-Франківськ набережна ім. В. Стефаника, 2,...                                 NaN   
Івано-Франківськ ДНЗ №20 Івано-Франківськ вул. Незалежності, 6                                              NaN   
                         Івано-Франківськ вулиця С.Петлюри, 21                                              NaN   
Івано-Франківськ ДНЗ №22 Івано-Франківськ вулиця Довга, 40а                                                 NaN   
...                                                                                                         ...   
Чернігів Школа №16       Чернігів, Шевченка, 93                                          Чернігів, Шевченка, 93   
Чернігів Школа №17       Чернігів, Малясова, 14                                          Чернігів, Малясова, 14   
Чернігів Школа №18       Чернігів, Гагаріна, 24                                

In [218]:
address_test[address_test.address2 == 'Невідомо']

,city_school,address,city_zamovnik_or_postavka,address2
227,Вінниця ДЮСШ №1,NaN,Невідомо,Невідомо
230,Вінниця ДЮСШ №5,NaN,Невідомо,Невідомо
231,Вінниця ДЮСШ №6,NaN,Невідомо,Невідомо
1880,Запоріжжя ДЮСШ №10,NaN,Невідомо,Невідомо
1881,Запоріжжя ДЮСШ №16,NaN,Невідомо,Невідомо
1882,Запоріжжя ДЮСШ №170,NaN,Невідомо,Невідомо
1883,Запоріжжя ДЮСШ №20,NaN,Невідомо,Невідомо
1884,Запоріжжя ДЮСШ №24,NaN,Невідомо,Невідомо
1885,Запоріжжя ДЮСШ №32,NaN,Невідомо,Невідомо
1886,Запоріжжя ДЮСШ №34,NaN,Невідомо,Невідомо


In [86]:
df_address['address2'].unique().tolist()

['Невідомо',
 ' м.Івано-Франківськ, Довга, 37',
 ' м.Івано-Франківськ, Богдана Лепкого, 9',
 ' м.Івано-Франківськ, Національної Гвардії, 13',
 ' м.Івано-Франківськ, Вовчинецька, 103',
 ' м.Івано-Франківськ, Тролейбусна, 7',
 ' м.Івано-Франківськ, Гната Хоткевича, 56',
 ' м.Івано-Франківськ, Дорошенка, 29',
 ' м.Івано-Франківськ, Шухевичів, 35',
 ' м.Івано-Франківськ, Сахарова, 36а',
 ' м.Івано-Франківськ, Івана Павла II, 24',
 ' м.Івано-Франківськ, Шевченка, 90',
 ' м.Івано-Франківськ, Хіміків, 1',
 ' м.Івано-Франківськ, 24 Серпня, 13',
 ' м.Івано-Франківськ, Шухевичів, 27а',
 ' м.Івано-Франківськ, Гоголя, 10',
 ' м.Івано-Франківськ, Франка, 14',
 ' м.Івано-Франківськ, Франка, 19',
 ' м.Івано-Франківськ, Джерельна, 2А',
 ' м.Івано-Франківськ, Грушевського, 16',
 ' м.Івано-Франківськ, Гетьмана Мазепи, 169, корп. 4',
 ' м.Житомир, Богунський район, Якіра, 26',
 ' м.Житомир, Вітрука, 55',
 ' м.Житомир, Каракульна, 15',
 ' м.Житомир, Черняховського, 17',
 ' м.Житомир, Корольовський район, 

In [84]:
df_address['address2'][df_address.isna().any(axis = 1)] = 'Невідомо'

In [154]:
df_address.address3.isna().sum()

2769

In [90]:
df_final = df_suma.merge(df_zakupki, on = 'address1')
df_final = df_final.merge(df_address, on = 'address1')
df_final.rename(columns={'Сума договору (згідно договору)':'Витрачена сума'}, inplace=True)
df_final.rename(columns={'address1':'Заклад'}, inplace=True)
df_final.rename(columns={'Закупівлі':'Витрачена сума'}, inplace=True)
df_final.rename(columns={'address2':'Адреса 2'}, inplace=True)
df_final.rename(columns={'address3':'Адреса 1'}, inplace=True)
df_final.rename(columns={'link':'Посилання'}, inplace=True)

df_final.to_csv('24_regions.csv', index = True, header=True)

In [91]:
pd.read_csv('24_regions.csv', index_col = 0)

,Заклад,Витрачена сума,Посилання,Адреса 2,address
0,Івано-Франківськ ДНЗ №1,743150.00,https://dozorro.org/tender/UA-2018-07-03-00170...,"вулиця Вовчинецька, 9",Невідомо
1,Івано-Франківськ ДНЗ №10,1430492.05,https://dozorro.org/tender/UA-2017-08-09-00125...,"вулиця Івана Миколайчука, 5",Невідомо
2,Івано-Франківськ ДНЗ №11,1140318.00,https://dozorro.org/tender/UA-2017-07-13-00164...,"вулиця Академіка Сахарова, 32а",Невідомо
3,Івано-Франківськ ДНЗ №12,370266.60,https://dozorro.org/tender/UA-2018-07-24-00047...,"вул. Довга, 4",Невідомо
4,Івано-Франківськ ДНЗ №14,852164.71,https://dozorro.org/tender/UA-2017-08-01-00225...,"вулиця М.Підгірянки,21",Невідомо
5,Івано-Франківськ ДНЗ №15,623348.00,https://dozorro.org/tender/UA-2017-09-27-00034...,"вулиця Нова, 19",Невідомо
6,Івано-Франківськ ДНЗ №16,1165309.00,https://dozorro.org/tender/UA-2017-11-23-00349...,"вулиця Короля Данила, 15а",Невідомо
7,Івано-Франківськ ДНЗ №17,96200.00,https://dozorro.org/tender/UA-2019-03-29-000986-b,"вулиця Вовчинецька, 210",Невідомо
8,Івано-Франківськ ДНЗ №18,2197095.90,https://dozorro.org/tender/UA-2018-08-08-00096...,"вул. Івана Павла ІІ, 14",Невідомо
9,Івано-Франківськ ДНЗ №19,390838.79,https://dozorro.org/tender/UA-2017-07-20-00155...,"вулиця Галицька, 101а",Невідомо
